In [17]:
#If you’re happy with the dataset, then load it
from datasets import load_dataset
from datasets import get_dataset_split_names
from datasets import load_dataset
from datasets import get_dataset_config_names
from transformers import AutoTokenizer
from datasets import load_dataset


from itertools import islice
from typing import Iterable, List, Tuple

import argparse
import numpy as np
import torch
import datasets
from datasets.dataset_dict import DatasetDict

from gensim.models.keyedvectors import KeyedVectors
from torch import nn
import random

In [ ]:
def batch(dataset: Iterable, batch_size: int) -> Iterable:
    jls_extract_var = """Creates batches from an iterable.
    
        Args:
            dataset (Iterable): Your dataset you want to batch given as an iterable (e.g. a list).
            batch_size (int): Your desired batch size
    
        Returns:
            Iterable: An iterable of tuples of size equal to batch_size.
    
        Example:
            >>> batches = batch([1,2, 3, 4, 5], 2)
            >>> print(list(batches))
            [(1, 2), (3, 4), (5,)]
        """
    iterable_dataset = iter(dataset)
    while True:
        chunk = list(islice(iterable_dataset, batch_size))
        if not chunk:  # when the dataset/chunk is empty break the while loop
            break
        yield chunk

In [18]:
# Load datasets
tifu_short_raw = load_dataset("reddit_tifu", "short", split="train")
tifu_long_raw = load_dataset("reddit_tifu", "long", split = "train")


Generating train split:   0%|          | 0/79740 [00:00<?, ? examples/s]

Dataset reddit_tifu downloaded and prepared to /home/coder/.cache/huggingface/datasets/reddit_tifu/short/1.1.0/1c73fb08807b54ec26b025829b2a3d90c6f7466dac20801c825571af9514c049. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/42139 [00:00<?, ? examples/s]

Dataset reddit_tifu downloaded and prepared to /home/coder/.cache/huggingface/datasets/reddit_tifu/long/1.1.0/1c73fb08807b54ec26b025829b2a3d90c6f7466dac20801c825571af9514c049. Subsequent calls will reuse this data.


In [19]:
# Establish model name:

model_name = "bert-base-uncased"

In [20]:
# Add decimal labels to upvote ratio
def add_columns(example):
    example["upvote_ratio"] = round(example["upvote_ratio"], 1)
    example["labels"] = int(example["upvote_ratio"] * 10)
    return example

tifu_short = tifu_short_raw.map(add_columns)

tifu_long = tifu_long_raw.map(add_columns)

  0%|          | 0/79740 [00:00<?, ?ex/s]

  0%|          | 0/42139 [00:00<?, ?ex/s]

In [ ]:
# Split into train, validation and test. MAKE THIS A FUNCTION
dict1 = tifu_short.train_test_split(test_size=0.1)

train_and_val = dict1["train"]
test_short = dict1["test"]

dict2 = train_and_val.train_test_split(test_size=0.2)

train_short = dict2["train"]
val_short = dict2["test"]


In [ ]:
print(tifu_short)
print(train_short)
print(val_short)
print(test_short)

In [ ]:
#print(len(train_short[0]["documents"]))

In [ ]:
# Tokenize # MAKE THIS AND OTHER PREPROCESSING A FUNCTION
tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenization(example):

    return tokenizer(example["documents"], padding = True, truncation= True, return_tensors = "pt")




train_input = train_short.map(tokenization, batched = True)
val_input = val_short.map(tokenization, batched = True)
test_input = test_short.map(tokenization, batched = True)

In [ ]:

# def tokenization(example):

#     return tokenizer(example["documents"])

# short_diagnostics = train_short.map(tokenization, batched = True)

In [ ]:
# len(short_diagnostics["input_ids"])

#a = short_diagnostics[1]
#len(a["input_ids"])

In [ ]:
# def add_length(example):
#     example["input_length"] = len(example["input_ids"])
#     return example

# short_diagnostics = short_diagnostics.map(add_length)

In [ ]:
train_input.save_to_disk("preprocessed/train_short")
val_input.save_to_disk("preprocessed/val_short")
test_input.save_to_disk("preprocessed/test_short")

In [ ]:
#train_batches = list(batch(train_cl_short["input_ids"], 32))

#train_labels = list(batch(train_cl_short["labels"], 32))


# Apparantly, this is where you would simply... throw the tokenizer at the model? Or use a dataloader?



In [ ]:
def tprint(thing):
    print(type(thing))


tprint(train_batches)
tprint(train_batches[0])
print(len(train_batches[0]))
tprint(train_batches[0][0])
tprint(train_batches[0][0][0])

tprint(train_labels)
tprint(train_labels[0])
print(len(train_labels[0]))
tprint(train_labels[0][0])

In [ ]:

def prepare_batch(token_ids, labels):

    token_id_tensor = torch.LongTensor(np.array(token_ids)) # Turn list of lists into np.array into torch.LongTensor
    labels = torch.LongTensor(np.array(labels))
    output = tuple([token_id_tensor, labels])
    return output

# its a list of lists!
# This can, if we want to, be turned into a np.array, and then a torch.tensor, and so on.

In [21]:
# Let's train a model!

from datasets import load_from_disk

train_cl_short = load_from_disk("../preprocessed/train_short")
val_cl_short = load_from_disk("../preprocessed/val_short")
test_cl_short = load_from_disk("../preprocessed/test_short")
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader


train_cl_short = train_cl_short.remove_columns(['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title'])
train_cl_short.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
train_dataloader = torch.utils.data.DataLoader(train_cl_short, batch_size=32)

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=11)
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
# dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

# dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)

Dataset({
    features: ['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 57412
})

In [22]:
nosave = 0
# Train model
epochs = 2
# batches = list(zip(train_batches, train_labels))

In [25]:
print("[INFO:] Training classifier...")

# Prepare validation dataset in a single batch
# For every epoch
k = 0
for epoch in range(epochs):
    ## for each batch
    k += 1
    for batch in train_dataloader:
        print("This is working for the ", k, "th time!")
        ## train on one batch
            # FORWARD PASS
        y = model(**batch)
            # Calculate loss
        loss = y.loss
            # backpropagation
        loss.backward()
            # take step, reset
        optimizer.step()
        optimizer.zero_grad()
        # Validate model
    #val_X = val_idx
    #val_y = model(val_X)
        # Calculate validation loss
    #val_loss = model.loss_fn(outputs=val_y, labels=val_tags)
        # If best_val_loss exists, see if it needs updating.
    #try:
    #    if best_val_loss > val_loss.item():
    #        best_val_loss = val_loss.item()
    #            # save the model if it is the best so far
    #        torch.save(model, f"models/{model_name}")
    #        nosave = 0
    #    else:
    #            # Increment nosave
    #        nosave += 1
        # Otherwise, create it and save the model.
    #except NameError:
    #    best_val_loss = val_loss.item()
    #    torch.save(model, f"models/{model_name}")
    #    # stop the training if patience is used up
    #if nosave > 5:
    #    print("Patience is up!")
    #    break
        # some print to see that it is running
        #if (epoch + 1) % 10 == 0: # Comment or uncomment depending on how many epochs you want printed.
    print(f"epoch: {epoch+1}, loss = {loss.item():.4f}, best validation loss = {best_val_loss:.4f}")

print("[INFO:] Finished traning!")


[INFO:] Training classifier...
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!
This is working for the  1 th time!


KeyboardInterrupt: 

PREPROCESSING DECISIONS:

1. Labels at every tenth percentile.

2. Max input sequence length (What do we do with those that exceed it? Do we truncate and hope for the best?)

Test the first: Truncation time!

3. Special characters, URLs, usernames, subreddit-names (Maybe the latter two are less relevant?).

In [ ]:
# Giant code dump from the Huggingface documentation.
# Don't even try to run this!

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)


encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")

print(encoding)
{'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

pt_batch = tokenizer(

    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],

    padding=True,

    truncation=True,

    max_length=512,

    return_tensors="pt",

)


from transformers import AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

pt_outputs = pt_model(**pt_batch)

from torch import nn

pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)

print(pt_predictions)
tensor([[0.0021, 0.0018, 0.0115, 0.2121, 0.7725],
        [0.2084, 0.1826, 0.1969, 0.1755, 0.2365]], grad_fn=<SoftmaxBackward0>)


pt_save_directory = "./pt_save_pretrained"

tokenizer.save_pretrained(pt_save_directory)

pt_model.save_pretrained(pt_save_directory)

pt_model = AutoModelForSequenceClassification.from_pretrained("./pt_save_pretrained")


from transformers import AutoModel

tokenizer = AutoTokenizer.from_pretrained(tf_save_directory)

pt_model = AutoModelForSequenceClassification.from_pretrained(tf_save_directory, from_tf=True)


from transformers import AutoConfig

my_config = AutoConfig.from_pretrained("distilbert-base-uncased", n_heads=12)

from transformers import AutoModel

my_model = AutoModel.from_config(my_config)

# Huggingface PyTorch Trainer
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(

    output_dir="path/to/save/folder/",

    learning_rate=2e-5,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=8,

    num_train_epochs=2,

)

# Preprocessing Class
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Preprocessed train and test data
train_dataset = dataset["train"]  # doctest: +SKIP

eval_dataset = dataset["eval"]  # doctest: +SKIP

# DataCollator to create examples
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

# Trainer!
from transformers import Trainer

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=dataset["train"],

    eval_dataset=dataset["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)  
# trainer.train()

